In [2]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Model Selection: SARIMA

SARIMA, or Seasonal Autoregressive Integrated Moving Average, is an extension of the ARIMA model that specifically includes components for seasonality. The justification for selecting SARIMA for the half-hourly energy demand time series in Victoria, Australia, is as follows:

### Data Characteristics
- **Stationarity**: The Augmented Dickey-Fuller (ADF) test confirmed that the time series is stationary, which is a prerequisite for ARIMA-based models.
- **Seasonality**: The time series decomposition and Fourier analysis revealed clear seasonal patterns, which SARIMA can model with its seasonal parameters.

### Model Capabilities
- **Flexibility**: SARIMA's flexibility allows it to model a broad class of seasonal time series.
- **Seasonal Parameters**: It includes seasonal autoregressive (P), differencing (D), and moving average (Q) parameters along with a period (s) parameter to explicitly model the seasonality observed in the data.
- **Interpretability**: SARIMA models provide interpretable parameters that correspond to the data's autocorrelations and partial autocorrelations, making them favorable for understanding the driving forces behind the time series.

### Predictive Performance
- **Accuracy**: SARIMA models are known for their accuracy in forecasting time series data with seasonal patterns.
- **Adaptability**: They can be tuned to adapt to changes in the time series' structure over time, accommodating evolving trends and seasonal effects.

Given these points, SARIMA is a well-suited model for forecasting the energy demand time series and will be the primary focus in the next phase of our analysis.


In [3]:
vic_data = pd.read_csv("vic_data.csv")

In [5]:
p, d, q = 1, 1, 1  # Non-seasonal orders
P, D, Q, s = 1, 1, 1, 48  # Seasonal orders (48 half-hours in a day for daily seasonality)

# Initialize the model
sarima_model = SARIMAX(vic_data['demand'], order=(p, d, q), seasonal_order=(P, D, Q, s))

# Fit the model
sarima_result = sarima_model.fit()

# Summary of the model
print(sarima_result.summary())

# Forecasting
n_periods = 48  # Let's forecast the next day (48 half-hour periods)
forecast = sarima_result.get_forecast(steps=n_periods)
forecast_df = forecast.conf_int()
forecast_df['Forecast'] = sarima_result.predict(start=forecast_df.index[0], end=forecast_df.index[-1])

# Plot the forecast alongside the data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(vic_data['demand'], label='Energy Demand')
plt.plot(forecast_df['Forecast'], label='Forecast')
plt.fill_between(forecast_df.index,
                 forecast_df.iloc[:, 0],
                 forecast_df.iloc[:, 1], color='k', alpha=0.2)
plt.legend()
plt.show()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.50960D+00    |proj g|=  1.10059D-01

At iterate    5    f=  5.43995D+00    |proj g|=  7.31330D-03

At iterate   10    f=  5.42859D+00    |proj g|=  1.88540D-02

At iterate   15    f=  5.39936D+00    |proj g|=  2.82116D-02

At iterate   20    f=  5.39869D+00    |proj g|=  6.12257D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     20     25      1     0     0   6.123D-06   5.399D+00
  F =   5.3986900743800623     

CONVERG

: 